# Engery Charts API
*Insert Lambda Function Name*

In [1]:
import json
import pandas as pd
import datetime as dt
import requests
import psycopg2
import os
import boto3

In [ ]:

ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

In [ ]:



def upload_dataframe_to_db(df, table_name, conn):
    cur = conn.cursor()
    
    # Replace invalid characters in column names and convert index to a column
    df = df.copy()
    df.index.name = 'timestamp'
    df.reset_index(inplace=True)
    df.columns = [col.lower().replace(" ", "_").replace("-", "_") for col in df.columns]
    
    # Generate CREATE TABLE query
    column_defs = ', '.join([
        f"{col} {'timestamp' if col == 'timestamp' else ('text' if col == 'unit' else 'float')}"
        for col in df.columns
    ])
    create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_defs});"
    cur.execute(create_query)

    # INSERT query
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
    
    # Convert NaN to None for SQL compatibility
    values = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
    cur.executemany(insert_query, values)
    
    cur.close()

def upload_dataframe_to_bucket(df, foldername):
    today = str(dt.datetime.today().date())

    s3 = boto3.client('s3')
    bucket_name = 'hslu-test-env-brunner'
    key = foldername + '/' + today + '.json'  # Key = path in the bucket

    data = df.reset_index().to_json(orient="records", date_format="iso")

    s3.put_object(
        Bucket=bucket_name,
        Key=key,
        Body=json.dumps(data),
        ContentType='application/json'
    )

def lambda_handler(event, context):
    date = dt.datetime.today().date()
    start_date = str(date-dt.timedelta(days=1))

    api_result_power = requests.get(f'https://api.energy-charts.info/public_power?country=ch&start={start_date}')
    api_result_prices = requests.get(f'https://api.energy-charts.info/price?bzn=CH&start={start_date}')

    api_response_power = api_result_power.json()
    api_response_prices = api_result_prices.json()

    api_df_power = pd.DataFrame(api_response_power["production_types"])
    api_df_power_t = pd.DataFrame(api_df_power['data'].tolist()).T
    api_df_power_t.columns = api_df_power['name']
    api_df_power_t.index = pd.to_datetime(api_response_power["unix_seconds"], unit='s')

    api_df_price = pd.DataFrame(api_response_prices["price"])
    api_df_price['Unit'] = api_response_prices["unit"]
    api_df_price.columns = ['Price', 'Unit']
    api_df_price.index = pd.to_datetime(api_response_prices["unix_seconds"], unit='s')

    try:
        print("Connecting to DB...")
        conn = psycopg2.connect(
            host=ENDPOINT,
            dbname=DB_NAME,
            user=USERNAME,
            password=PASSWORD
        )
        conn.set_session(autocommit=True)
        print("Connection successful.")
                
        # Upload DataFrames
        upload_dataframe_to_db(api_df_power_t, "power_data", conn)
        upload_dataframe_to_db(api_df_price, "price_data", conn)

        upload_dataframe_to_bucket(api_df_power_t, "power")
        upload_dataframe_to_bucket(api_df_price, "price")

        conn.close()
        print("Data uploaded and connection closed.")

    except Exception as e:
        print("Error:", e)

    

    


    return {
        'statusCode': 200,
        'body': json.dumps("Inport succesfully")
    }
